## Differential Geometry

In [1]:
from sympy import symbols, pi, sqrt, atan2, cos, sin
from sympy.diffgeom import Manifold, Patch, CoordSystem

# define a manifold and a coordinate patch
manifold = Manifold('M', 2)
patch = Patch('P', manifold)

x, y = symbols('x y', real=True)
r, theta = symbols('r theta', nonnegative=True)

# Define the transformations from cartesian to polar and vice versa
relation_dict = {
('Car2D', 'Pol'): [(x, y), (sqrt(x**2 + y**2), atan2(y, x))],
('Pol', 'Car2D'): [(r, theta), (r*cos(theta), r*sin(theta))]
}


# Define the two coordinate systems
Car2D = CoordSystem('Car2D', patch, (x, y), relation_dict)
Pol = CoordSystem('Pol', patch, (r, theta), relation_dict)

In [2]:
# This simply reproduces the transformations we defined

Car2D.transformation(Pol)

Lambda((x, y), Matrix([
[sqrt(x**2 + y**2)],
[      atan2(y, x)]]))

In [3]:
Pol.transformation(Car2D)

Lambda((r, theta), Matrix([
[r*cos(theta)],
[r*sin(theta)]]))

In [4]:
# Now we get something new. The Jacobians
Pol.jacobian(Car2D)


Matrix([
[cos(theta), -r*sin(theta)],
[sin(theta),  r*cos(theta)]])

In [5]:
Car2D.jacobian(Pol)

Matrix([
[x/sqrt(x**2 + y**2), y/sqrt(x**2 + y**2)],
[   -y/(x**2 + y**2),     x/(x**2 + y**2)]])

In [6]:
# We can convert functional expressions from 2D to polar and vice versa

x, y = Car2D.symbols
x.rewrite(Pol)

r*cos(theta)

In [7]:
(x**2 + 1/y).rewrite(Pol)

r**2*cos(theta)**2 + 1/(r*sin(theta))

In [8]:
from sympy import pi
from sympy.diffgeom import Point
from sympy.diffgeom.rn import R2 # The R^2 Manifold
from sympy.diffgeom.rn import R2_r, R2_p # coordinate patches in cartesian and polar coordinates

rho, theta = R2_p.symbols

In [9]:
p = Point(R2_p, [rho, 3*pi/4])
p.coords()

Matrix([
[   rho],
[3*pi/4]])

In [10]:
p.coords(R2_r)

Matrix([
[-sqrt(2)*rho/2],
[ sqrt(2)*rho/2]])

## Scalar Fields

In [11]:
from sympy import Function
from sympy.diffgeom import BaseScalarField

rho, _ = R2_p.symbols
fx, fy = R2_r.base_scalars()
ftheta = BaseScalarField(R2_r, 1)

In [12]:
#evaluate field at a point

point = R2_p.point([rho, 0])
fx(point)

rho

In [13]:
fy(point)

0

In [14]:
# Evaluate a function of the field coordinates at a point
(fx**2+fy**2).rcall(point)

rho**2

## Vector Fields

In [15]:
from sympy.diffgeom import BaseVectorField
from sympy import pprint

x, y = R2_r.symbols
rho, theta = R2_p.symbols
fx, fy = R2_r.base_scalars()
point_p = R2_p.point([rho, theta])
point_r = R2_r.point([x, y])

In [16]:
# A vector field is an operator taking a scalar field and returning a directional derivative (which is also a scalar field). 
# A base vector field is the same type of operator, 
# however the derivation is specifically done with respect to a chosen coordinate.

g = Function('g')
scalar_field = g(fx, fy)

In [17]:
v = BaseVectorField(R2_r, 1)
pprint(v(scalar_field))

⎛d          ⎞│   
⎜──(g(x, ξ))⎟│   
⎝dξ         ⎠│ξ=y


In [18]:
# Commutator of two vector fields

from sympy.diffgeom import Commutator
from sympy import simplify

fx, fy = R2_r.base_scalars()
e_x, e_y = R2_r.base_vectors()
e_r, e_theta = R2_p.base_vectors()

In [19]:
Commutator(e_x, e_y)

0

In [20]:
Commutator(e_x, e_r)

Commutator(e_x, e_rho)

In [21]:
c_xr = Commutator(e_x, e_r)
c_xr(fy**2)

-2*cos(theta)*y**2/(x**2 + y**2)

In [22]:
c_xr(fy**2+fx**2)

2*sin(theta)*x*y/(x**2 + y**2) - 2*cos(theta)*y**2/(x**2 + y**2)

In [23]:
# Tensors and forms

from sympy.diffgeom import TensorProduct

e_x, e_y = R2_r.base_vectors()

In [24]:
e_x

e_x

In [25]:
TensorProduct(e_x, e_y)

TensorProduct(e_x, e_y)

In [26]:
dx, dy = R2_r.base_oneforms()

In [27]:
dy

dy

In [28]:
TensorProduct(dx, dy)

TensorProduct(dx, dy)

In [29]:
TensorProduct(dx, dy)(e_x, e_y)

1

## Lie Derivative

In [30]:
from sympy.diffgeom import *

M = Manifold("M", 5)
P = Patch("P", M)

x, y, L, u, v = symbols('x y L u v', real=True)

coord = CoordSystem("coord", P, [x, y, L, u, v])

x, y, L, u, v = coord.coord_functions()

In [31]:
 # Matrices are currently not supported, you have to express them as linear combination of base vector fields:

e_x, e_y, e_L, e_u, e_v = coord.base_vectors()

In [32]:
# define a vector as a sum of components and base vectors:

expr = (x + u)*e_x + (y + v)*e_y + L*e_L
expr

(x + u)*e_x + (y + v)*e_y + L*e_L

In [33]:
LieDerivative(expr, sqrt(L**2 + (y - x)**2))

(-x + y)*(y + v)/sqrt((-x + y)**2 + L**2) + (x - y)*(x + u)/sqrt((-x + y)**2 + L**2) + L**2/sqrt((-x + y)**2 + L**2)